In [39]:
import pandas as pd
import pickle

import os

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric


ModuleNotFoundError: No module named 'evidently.metric_packs'

In [13]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

In [29]:
import joblib
import numpy as np
import scipy.sparse as sp
from sklearn.metrics import accuracy_score, classification_report

# Cargar el modelo
model = joblib.load('model.pkl')

# Cargar datos escalados
load_dir = '../data/gold'
X_train_scaled_npz = np.load(os.path.join(load_dir, 'X_train_scaled.npz'))
X_test_scaled_npz = np.load(os.path.join(load_dir, 'X_test_scaled.npz'))

# Convertir datos escalados de formato sparse a csr_matrix
X_train_scaled_sparse = sp.csr_matrix((X_train_scaled_npz['data'], X_train_scaled_npz['indices'], X_train_scaled_npz['indptr']),
                                      shape=X_train_scaled_npz['shape'])
X_test_scaled_sparse = sp.csr_matrix((X_test_scaled_npz['data'], X_test_scaled_npz['indices'], X_test_scaled_npz['indptr']),
                                     shape=X_test_scaled_npz['shape'])

# Cargar etiquetas
y_train = joblib.load(os.path.join(load_dir, 'y_train.pkl'))
y_test = joblib.load(os.path.join(load_dir, 'y_test.pkl'))

# Seleccionar subconjuntos
X_train_scaled_subset_sparse = X_train_scaled_sparse[:8000]
y_train_subset = y_train[:8000]

X_test_scaled_subset_sparse = X_test_scaled_sparse[:2000]
y_test_subset = y_test[:2000]

# Realizar predicciones con el modelo Random Forest
train_preds = model.predict(X_train_scaled_subset_sparse)
val_preds = model.predict(X_test_scaled_subset_sparse)

# Evaluar el modelo
train_accuracy = accuracy_score(y_train_subset, train_preds)
print(f'Accuracy en entrenamiento: {train_accuracy:.2f}')

val_accuracy = accuracy_score(y_test_subset, val_preds)
print(f'Accuracy en prueba: {val_accuracy:.2f}')

print('Reporte de clasificación para el conjunto de prueba:')
print(classification_report(y_test_subset, val_preds))


Accuracy en entrenamiento: 1.00
Accuracy en prueba: 1.00
Reporte de clasificación para el conjunto de prueba:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1995
           1       0.00      0.00      0.00         5

    accuracy                           1.00      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       1.00      1.00      1.00      2000



/home/maldu/.local/share/virtualenvs/fraud_detection-I9haicwR/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maldu/.local/share/virtualenvs/fraud_detection-I9haicwR/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maldu/.local/share/virtualenvs/fraud_detection-I9haicwR/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [30]:
numerical_features = df.select_dtypes(include=['int', 'float'])
categorical_features = df.select_dtypes(include=['category'])

In [31]:
column_mapping = ColumnMapping(
    target = None,
    prediction = 'pred',
    numerical_features=numerical_features,
    categorical_features= categorical_features
)

In [32]:
report = Report(metrics=[
    ColumnDriftMetric(column_name = 'isFraud'),
    DatasetDriftMetric(),
    DatasetMissingValuesMetric()
    
    ])

In [37]:
column_mapping = {"target": "target"}
report = Report(metrics=[DataDriftMetricPack()])

X_train_df = pd.DataFrame.sparse.from_spmatrix(X_train_scaled_subset_sparse)
X_test_df = pd.DataFrame.sparse.from_spmatrix(X_test_scaled_subset_sparse)


report.run(reference_data=X_train_df, current_data=X_test_df, column_mapping=column_mapping)

NameError: name 'DataDriftMetricPack' is not defined

In [ ]:
report.show(mode='inline')


guardar train_data y val_data en cache

In [ ]:
report.run(reference_data=train_data, current_data= val_data, column_mapping=column_mapping)
report.show(mode='inline')

In [ ]:
result = report.as_dict()
result

In [ ]:
# prediction drift
result['metrics'][0]['result']['drift_score']

In [ ]:
#number of drifted columns
result['metrics'][0]['result']

In [ ]:
#share of missing values
result['metrics'][2]['result']['current']